In [ ]:
### Datasets

https://universe.roboflow.com/rlggypface/face-detection-zspaa

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4065, done.
remote: Counting objects: 100% (4065/4065), done.
remote: Compressing objects: 100% (3088/3088), done.
remote: Total 4065 (delta 1186), reused 1960 (delta 917), pack-reused 0
Receiving objects: 100% (4065/4065), 54.71 MiB | 14.23 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cp /content/models/research/object_detection/packages/tf2/setup.py /content/models/research/

In [ ]:
%%bash
cd models/research
pip install .

In [ ]:
# !pip install Cython -y
!pip install -U --pre tensorflow=="2.13.0"
!pip install tf_slim
!pip install pycocotools

In [ ]:
!pip uninstall Cython -y

Found existing installation: Cython 3.0.6
Uninstalling Cython-3.0.6:
  Successfully uninstalled Cython-3.0.6


In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

2023-12-03 03:20:18.488853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 03:20:19.497223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-03 03:20:22.462198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 03:20:22.943790: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returnin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp /content/drive/MyDrive/ssd-mobilenet/labeled_data/images_labeled_18nov23.zip /content/images.zip
!unzip /content/drive/MyDrive/style-mate/datasets/face-detection.v1i.tfrecord.zip -d /content/

Archive:  /content/drive/MyDrive/style-mate/datasets/face-detection.v1i.tfrecord.zip
 extracting: /content/README.dataset.txt  
 extracting: /content/README.roboflow.txt  
   creating: /content/train/
 extracting: /content/train/face.tfrecord  
 extracting: /content/train/face_label_map.pbtxt  
   creating: /content/valid/
 extracting: /content/valid/face.tfrecord  
 extracting: /content/valid/face_label_map.pbtxt  


In [ ]:
# !unzip /content/images.zip
# !unzip /content/face-detection.v1i.tfrecord.zip -d /content/
# !mkdir /content/images
# !unzip -j -o /content/images.zip -d /content/images/all
# # !unzip -o /content/images.zip
# !mkdir /content/images/train; mkdir /content/images/validation; mkdir /content/images/test

In [ ]:
import os
import xml.etree.ElementTree as ET

def has_annotation(xml_path):
    try:
        # Baca file XML
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Ganti ini dengan XPath atau metode lain untuk menentukan apakah XML berisi anotasi yang diinginkan
        # Di sini kita mengasumsikan bahwa keberadaan elemen "object" menunjukkan adanya anotasi
        return len(root.findall(".//object")) > 0
    except Exception as e:
        # Tangani kesalahan saat membaca atau memproses file XML
        print(f"Error processing {xml_path}: {e}")
        return False

def remove_unannotated_images(image_dir, xml_dir):
    # Ambil daftar nama file gambar dengan ekstensi .jpg dari direktori gambar
    image_list = [file for file in os.listdir(image_dir) if file.endswith(".jpeg")]

    # Iterasi melalui setiap file gambar
    for img_name in image_list:
        img_path = os.path.join(image_dir, img_name)

        # Dapatkan nama file XML sesuai dengan nama file gambar
        xml_name = os.path.splitext(img_name)[0] + ".xml"
        xml_path = os.path.join(xml_dir, xml_name)

        # Periksa apakah file XML sesuai dengan file gambar
        if os.path.exists(xml_path):
            # Periksa apakah file XML memiliki anotasi
            if not has_annotation(xml_path):
                # Hapus file gambar jika tidak memiliki anotasi
                os.remove(img_path)
                print(f"Removed---->: {img_name}")
        else:
            os.remove(img_path)
            print(f"Removed---->: {img_name}")

# Tentukan direktori gambar (.jpg)
image_directory = '/content/images/'

# Tentukan direktori label XML
xml_directory = '/content/images/'

# Panggil fungsi untuk menghapus gambar tanpa anotasi
# remove_unannotated_images('/content/images/all', '/content/images/all')
remove_unannotated_images('/content/images/train', '/content/images/train')
remove_unannotated_images('/content/images/validation', '/content/images/validation')


Removed---->: img17nov_0_8651.jpeg
Removed---->: img17nov_0_9811.jpeg
Removed---->: img17nov_0_2776.jpeg


In [ ]:
!wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/train_val_test_split.py

--2023-11-19 04:32:59--  https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/train_val_test_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2803 (2.7K) [text/plain]
Saving to: ‘train_val_test_split.py’

train_val_test_spli 100%[===================>]   2.74K  --.-KB/s    in 0s      

2023-11-19 04:33:00 (46.8 MB/s) - ‘train_val_test_split.py’ saved [2803/2803]



In [ ]:
!python /content/train_val_test_split.py

Total images: 2276
Images moving to train: 1934
Images moving to validation: 341
Images moving to test: 1


In [ ]:
%%bash
cat <<EOF >> /content/labelmap.txt
face
EOF

In [ ]:
# Download data conversion scripts
! wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/create_csv.py
! wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/create_tfrecord.py

--2023-11-20 22:38:52--  https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/create_csv.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348 (1.3K) [text/plain]
Saving to: ‘create_csv.py’

create_csv.py       100%[===================>]   1.32K  --.-KB/s    in 0s      

2023-11-20 22:38:52 (55.4 MB/s) - ‘create_csv.py’ saved [1348/1348]

--2023-11-20 22:38:52--  https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/create_tfrecord.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githu

In [ ]:
%%bash
cd /content/
python3 create_csv.py
python3 create_tfrecord.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
python3 create_tfrecord.py --csv_input=images/validation_labels.csv --labelmap=labelmap.txt --image_dir=images/validation --output_path=val.tfrecord

Successfully converted xml to csv.
Successfully converted xml to csv.
Successfully created the TFRecords: /content/train.tfrecord
Successfully created the TFRecords: /content/val.tfrecord


2023-11-20 22:38:55.942019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 22:39:00.359869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-20 22:39:10.934571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 22:39:13.431115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!unzip /content/ball_tf_record.zip

Archive:  /content/ball_tf_record.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/circle.tfrecord    
 extracting: test/circle_label_map.pbtxt  
   creating: train/
 extracting: train/circle.tfrecord   
 extracting: train/circle_label_map.pbtxt  
   creating: valid/
 extracting: valid/circle.tfrecord   
 extracting: valid/circle_label_map.pbtxt  


In [ ]:
# train_record_fname = '/content/train.tfrecord'
# val_record_fname = '/content/val.tfrecord'
# label_map_pbtxt_fname = '/content/labelmap.pbtxt'
train_record_fname = '/content/train/face.tfrecord'
val_record_fname = '/content/valid/face.tfrecord'
label_map_pbtxt_fname = '/content/train/face_label_map.pbtxt'

In [ ]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    # The centernet model isn't working as of 9/10/22
    #'centernet-mobilenet-v2': {
    #    'model_name': 'centernet_mobilenetv2fpn_512x512_coco17_od',
    #    'base_pipeline_file': 'pipeline.config',
    #    'pretrained_checkpoint': 'centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz',
    #}
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
num_steps = 5000
batch_size = 32

In [ ]:
# Set file locations and get number of classes for config file
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)


Total classes: 1


In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /content/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .03', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


/content/models/mymodel
writing custom configuration file


In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/models/mymodel/pipeline_file.config'
# model_dir = '/content/training/'
model_dir = '/content/drive/MyDrive/style-mate/training/'

In [ ]:
# Run training!
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

In [ ]:
# Make a directory to store the trained TFLite model
OUTPUT_MODEL_DIR='custom_model_lite_face_detection'
!mkdir $OUTPUT_MODEL_DIR
output_directory = '/content/' + OUTPUT_MODEL_DIR

# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = model_dir

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}


In [ ]:
# Convert exported graph file into TFLite model file
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/' + OUTPUT_MODEL_DIR + '/saved_model')
tflite_model = converter.convert()

with open('/content/' + OUTPUT_MODEL_DIR + '/detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Move labelmap and pipeline config files into TFLite model folder and zip it up
!cp /content/labelmap.txt /content/$OUTPUT_MODEL_DIR/
!cp /content/labelmap.pbtxt /content/$OUTPUT_MODEL_DIR/
!cp /content/models/mymodel/pipeline_file.config /content/$OUTPUT_MODEL_DIR

%cd /content

ZIP_FILENAME='face_detection_style_mate.zip'

!zip -r $ZIP_FILENAME $OUTPUT_MODEL_DIR

cp: cannot stat '/content/labelmap.pbtxt': No such file or directory
/content
  adding: custom_model_lite_face_detection/ (stored 0%)
  adding: custom_model_lite_face_detection/pipeline_file.config (deflated 66%)
  adding: custom_model_lite_face_detection/saved_model/ (stored 0%)
  adding: custom_model_lite_face_detection/saved_model/assets/ (stored 0%)
  adding: custom_model_lite_face_detection/saved_model/saved_model.pb (deflated 91%)
  adding: custom_model_lite_face_detection/saved_model/fingerprint.pb (stored 0%)
  adding: custom_model_lite_face_detection/saved_model/variables/ (stored 0%)
  adding: custom_model_lite_face_detection/saved_model/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: custom_model_lite_face_detection/saved_model/variables/variables.index (deflated 78%)
  adding: custom_model_lite_face_detection/detect.tflite (deflated 9%)
  adding: custom_model_lite_face_detection/labelmap.txt (stored 0%)


In [ ]:
from google.colab import files

files.download('/content/' + ZIP_FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# !cp -r /content/training /content/drive/MyDrive/ssd-mobilenet/

In [ ]:
# !zip checkpoint_roboflow_ball.zip -r /content/training

  adding: content/training/ (stored 0%)
  adding: content/training/ckpt-7.index (deflated 82%)
  adding: content/training/ckpt-9.data-00000-of-00001 (deflated 8%)
  adding: content/training/ckpt-5.index (deflated 82%)
  adding: content/training/ckpt-6.index (deflated 82%)
  adding: content/training/ckpt-5.data-00000-of-00001 (deflated 7%)
  adding: content/training/ckpt-11.data-00000-of-00001 (deflated 8%)
  adding: content/training/ckpt-10.index (deflated 82%)
  adding: content/training/ckpt-8.data-00000-of-00001 (deflated 8%)
  adding: content/training/checkpoint (deflated 76%)
  adding: content/training/ckpt-7.data-00000-of-00001 (deflated 8%)
  adding: content/training/ckpt-9.index (deflated 82%)
  adding: content/training/ckpt-11.index (deflated 82%)
  adding: content/training/ckpt-10.data-00000-of-00001 (deflated 8%)
  adding: content/training/train/ (stored 0%)
  adding: content/training/train/events.out.tfevents.1701064210.1042d5521955.3194.0.v2 (deflated 0%)
  adding: content/

In [ ]:
# !cp checkpoint_roboflow_ball.zip /content/drive/MyDrive/ssd-mobilenet/

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='style-mate' \
    --saved_model_tags=serve \
    /content/custom_model_lite_face_detection/saved_model \
    /mobilenet/web_model


In [ ]:
import tensorflow as tf

In [ ]:
model = tf.saved_model.load('/content/custom_model_lite_face_detection/saved_model/')

In [ ]:
!zip /content/mobilenet_style_mate.zip -r /mobilenet

In [ ]:
from google.colab import files

files.download('/content/mobilenet_style_mate.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>